# Online Inference

This tutorial shows how to use trained PyTorch, TensorFlow, and ONNX (format) models, written in Python, directly in HPC workloads written in Fortran, C, C++ and Python.

The example simulation here is written in Python for brevity, however, the inference API in SmartRedis is the same (besides extra parameters for compiled langauges) across all clients. 


## Installing the ML backends

In order to use the `Orchestrator` database as an inference engine, the Machine Learning (ML) backends need to be built and supplied to the database at runtime. 

To check which backends are built, a simple helper function is available in SmartSim as shown below.

In [ ]:
## Installing the ML backends
from smartsim._core.utils.helpers import installed_redisai_backends
print(installed_redisai_backends())

For this tutorial, we have already built the pyTorch, Tensorflow, and ONNX backends. Users following the install directions, generally will have to install these backends on a user-by-user basis. For larger teams, CrayLabs will help setup your system so that the backends do not have to be built by each user.

## Starting the Database for Inference

SmartSim performs online inference by using the SmartRedis clients to call into the
Machine Learning (ML) runtimes linked into the Orchestrator database. The Orchestrator
is the name in SmartSim for a Redis or KeyDB database with a RedisAI module built
into it with the ML runtimes.

Therefore, to perform inference, you must first create an Orchestrator database and
launch it. There are two methods to couple the database to your application in
order to add inference capability to your application.
 - standard (not co-located)
 - co-located
 
`standard` mode launches an optionally clustered (across many compute hosts) database instance
that can be treated as a single storage device for many clients (possibly the many ranks
of an MPI program) where there is a single address space for keys across all hosts.

`co-located` mode launches a orchestrator instance on each compute host used by a,
possibly distributed, application. each instance contains their own address space
for keys. In SmartSim, `Model` instances can be launched with a co-located orchetrator
through `Model.colocate_db`. Co-located `Model`s are used for highly scalable
inference where global aggregations aren't necessary for inference.

The code below launches the `Orchestrator` database using the `standard` deployment
method.

In [ ]:
# some helper libraries for the tutorial
import io
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import logging
import numpy as np

# import smartsim and smartredis
from smartredis import Client
from smartsim import Experiment

In [ ]:
exp = Experiment("Inference-Tutorial", launcher="local")

In [ ]:
db = exp.create_database(port=6780, interface="lo")
exp.start(db)

## Using PyTorch

The Orchestrator supports both [PyTorch](https://pytorch.org/)
models and [TorchScript](https://pytorch.org/docs/stable/jit.html) functions and scripts
in PyTorch.

Below, the code is shown to create, jit-trace (prepare for inference), set,
and call a PyTorch Convolutional Neural Network (CNN) with SmartSim and SmartRedis

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


To set a PyTorch model, we create a function to "jit-trace" the model
and save it to a buffer in memory.

If you aren't familiar with the concept of tracing, take a look at the
Torch documentation for [trace](https://pytorch.org/docs/stable/generated/torch.jit.trace.html#torch.jit.trace)


In [ ]:
# Initialize an instance of our CNN model
n = Net()
n.eval()

# prepare a sample input to trace on (random noise is fine)
example_forward_input = torch.rand(1, 1, 28, 28)

def create_torch_model(torch_module, example_forward_input):

    # perform the trace of the nn.Module.forward() method
    module = torch.jit.trace(torch_module, example_forward_input)

    # save the traced module to a buffer
    model_buffer = io.BytesIO()
    torch.jit.save(module, model_buffer)
    return model_buffer.getvalue()

traced_cnn = create_torch_model(n, example_forward_input)

Lastly, we use the SmartRedis Python client to

1. Connect to the database
2. Put a batch of 20 tensors into the database  (``put_tensor``)
3. Set the Torch model in the database (``set_model``)
4. Run the model on the batch of tensors (``run_model``)
5. Retrieve the result (``get_tensor``)


In [ ]:
client = Client(address=db.get_address()[0], cluster=False)

client.put_tensor("input", torch.rand(20, 1, 28, 28).numpy())

# put the PyTorch CNN in the database in GPU memory
client.set_model("cnn", traced_cnn, "TORCH", device="CPU")

# execute the model, supports a variable number of inputs and outputs
client.run_model("cnn", inputs=["input"], outputs=["output"])

# get the output
output = client.get_tensor("output")
print(f"Prediction: {output}")


As we gave the CNN random noise, the predictions reflect that.

If running on CPU, be sure to change the argument in the ``set_model`` call
above to ``CPU``.

## Using TorchScript

In addition to PyTorch models, TorchScript scripts and functions can be set in the
Orchestrator database and called from any of the SmartRedis languages. Functions
can be set in the database in Python prior to application launch and then used
directly in Fortran, C, and C++ simulations.

The example below uses the TorchScript Singular Value Decomposition (SVD) function.
The function set in side the database and then called with a random input
tensor.


In [ ]:
def calc_svd(input_tensor):
    # svd function from TorchScript API
    return input_tensor.svd()

In [ ]:
# connect a client to the database
client = Client(address=db.get_address()[0], cluster=False)

# test the SVD function
tensor = np.random.randint(0, 100, size=(5, 3, 2)).astype(np.float32)
client.put_tensor("input", tensor)
client.set_function("svd", calc_svd)
client.run_script("svd", "calc_svd", ["input"], ["U", "S", "V"])
U = client.get_tensor("U")
S = client.get_tensor("S")
V = client.get_tensor("V")
print(f"U: {U}\n\n, S: {S}\n\n, V: {V}\n")


In [ ]:
## TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
tf.get_logger().setLevel(logging.ERROR)

# create a simple Fully connected network in Keras
model = keras.Sequential(
    layers=[
        keras.layers.InputLayer(input_shape=(28, 28), name="input"),
        keras.layers.Flatten(input_shape=(28, 28), name="flatten"),
        keras.layers.Dense(128, activation="relu", name="dense"),
        keras.layers.Dense(10, activation="softmax", name="output"),
    ],
    name="FCN",
)

# Compile model with optimizer
model.compile(optimizer="adam",
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])

### Setting TensorFlow and Keras Models

After a model is created (trained or not), the graph of the model is
frozen and saved to file so the client method `client.set_model_from_file`
can load it into the database.

SmartSim includes a utility to freeze the graph of a TensorFlow or Keras model in
`smartsim.ml.tf`. To use TensorFlow or Keras in SmartSim, specify
`TF` as the argument for *backend* in the call to `client.set_model` or
`client.set_model_from_file`.

Note that TensorFlow and Keras, unlike the other ML libraries supported by
SmartSim, requires an `input` and `output` argument in the call to
`set_model`. These arguments correspond to the layer names of the
created model. The `smartsim.ml.tf.freeze_model` utility
returns these values for convenience as shown below.

In [ ]:
from smartsim.ml.tf import freeze_model

# SmartSim utility for Freezing the model and saving it to a file.
model_path, inputs, outputs = freeze_model(model, os.getcwd(), "fcn.pb")

# use the same client we used for PyTorch to set the TensorFlow model
# this time the method for setting a model from a saved file is shown. 
# TensorFlow backed requires named inputs and outputs on graph
# this differs from PyTorch and ONNX.
client.set_model_from_file(
    "keras_fcn", model_path, "TF", device="CPU", inputs=inputs, outputs=outputs
)

# put random random input tensor into the database
input_data = np.random.rand(1, 28, 28).astype(np.float32)
client.put_tensor("input", input_data)

# run the Fully Connected Network model on the tensor we just put
# in and store the result of the inference at the "output" key
client.run_model("keras_fcn", "input", "output")

# get the result of the inference
pred = client.get_tensor("output")
print(pred)

## Using ONNX

ONNX is a standard format for representing models. A number of different Machine Learning
Libraries are supported by ONNX and can be readily used with SmartSim.

Some popular ones are:


- [Scikit-learn](https://scikit-learn.org)
- [XGBoost](https://xgboost.readthedocs.io)
- [CatBoost](https://catboost.ai)
- [LightGBM](https://lightgbm.readthedocs.io/en/latest/)
- [libsvm](https://www.csie.ntu.edu.tw/~cjlin/libsvm/)


As well as some that are not listed. There are also many tools to help convert
models to ONNX.

- [onnxmltools](https://github.com/onnx/onnxmltools)
- [skl2onnx](https://github.com/onnx/sklearn-onnx/)
- [tensorflow-onnx](https://github.com/onnx/tensorflow-onnx/)


And PyTorch has its own converter.

Currently the ONNX backend only works on Linux, but MacOS support will be added in the future.

Below are some examples of a few models in [Scikit-learn](https://scikit-learn.org)
that are converted into ONNX format for use with SmartSim. To use ONNX in SmartSim, specify
`ONNX` as the argument for *backend* in the call to `client.set_model` or
`client.set_model_from_file`

### Scikit-Learn K-means Cluster


K-means clustering is an unsupervised ML algorithm. It is used to categorize data points
into functional groups ("clusters"). Scikit Learn has a built in implementation of K-means clustering
and it is easily converted to ONNX for use with SmartSim through 
[skl2onnx.to_onnx](http://onnx.ai/sklearn-onnx/auto_examples/plot_convert_syntax.html)

Since the KMeans model returns two outputs, we provide the `client.run_model` call
with two `output` key names.


In [ ]:
from skl2onnx import to_onnx
from sklearn.cluster import KMeans

In [ ]:

X = np.arange(20, dtype=np.float32).reshape(10, 2)
tr = KMeans(n_clusters=2)
tr.fit(X)

# save the trained k-means model in memory with skl2onnx
kmeans = to_onnx(tr, X, target_opset=11)
model = kmeans.SerializeToString()

# random input data
sample = np.arange(20, dtype=np.float32).reshape(10, 2)

# use the same client from TensorFlow and Pytorch examples.
client.put_tensor("input", sample)
client.set_model("kmeans", model, "ONNX", device="CPU")
client.run_model("kmeans", inputs="input", outputs=["labels", "transform"])

print(client.get_tensor("labels"))


### Scikit-Learn Random Forest

The Random Forest example uses the Iris dataset from Scikit Learn to train a
RandomForestRegressor. As with the other examples, the skl2onnx function
`skl2onnx.to_onnx` is used to convert the model to ONNX format.


In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [ ]:
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, _ = train_test_split(X, y, random_state=13)
clr = RandomForestRegressor(n_jobs=1, n_estimators=100)
clr.fit(X_train, y_train)

rf_model = to_onnx(clr, X_test.astype(np.float32), target_opset=11)

sample = np.array([[6.4, 2.8, 5.6, 2.2]]).astype(np.float32)
model = rf_model.SerializeToString()

client.put_tensor("input", sample)
client.set_model("rf_regressor", model, "ONNX", device="CPU")
client.run_model("rf_regressor", inputs="input", outputs="output")
print(client.get_tensor("output"))


In [ ]:
exp.stop(db)

In [ ]:
exp.summary(format="html")